In [2]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def circuit(params):
    """The quantum circuit that you will differentiate!

    Args:
        params (list(float)): The parameters for gates in the circuit
    
    Returns:
        (numpy.array): An expectation value. 
    """
    qml.broadcast(qml.Hadamard, wires=range(3), pattern="single")
    qml.CRX(params[0], [1, 2])
    qml.CRY(params[1], [0, 1])
    qml.CRZ(params[2], [2, 0])
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1) + qml.PauliX(2))

'''
def circuit2(params, shifts, coeffs):    

    gradient = np.zeros_like(params)
    
    for i in range(len(params)):
        # Put your code here
        shifted = params.copy()
        shifted2 = params.copy()
        shifted3 = params.copy()
        shifted4 = params.copy()
        shifts2 = shifts.copy()
        coeffs2 = coeffs.copy()
        print(f'original param: {shifted}')
        shifted[i] = shifted[i] + shifts2[0]
        shifted2[i] = shifted2[i] - shifts2[0]
        
        shifted3[i] = shifted3[i] + shifts2[1]
        shifted4[i] = shifted4[i] - shifts2[1]
        print(f'modified param s1: {shifted}')
        print(f'modified param -s1: {shifted2}')
        print(f'modified param s2: {shifted3}')
        print(f'modified param -s2: {shifted4}')
        f1 = circuit(shifted)
        f2 = circuit(shifted2)
        forward = f1 - f2
    
        b1 = circuit(shifted3)
        b2 = circuit(shifted4)
        back = b1 - b2
    
        gradient[i] = coeffs[0]*forward - coeffs[1]*back
        print(f'gradient[{i}]: {gradient[i]}\n')
        
        
    return np.round_(gradient, decimals=5).tolist()


def error(params, shifts, coeffs):
    
    error = 0
    answer = [0.08144, -0.33706, -0.37944]
    gradient = circuit2(params, shifts, coeffs)
    
    for i in range(len(answer)):
        error += np.abs(answer[i] - gradients[i])
    print(f'error: {error}')
    return error

def train_parameters(params, shifts, coeffs):

    epochs = 1000
    lr = 0.01

    grad = qml.grad(error, argnum=1)
    shifts = [1.5,1.05]
    coeffs = [1.55,1] 
    
    for epoch in range(epochs):
        print(f'{epoch} : {shifts}')
        shifts -= lr * grad(params, shifts, coeffs)

    return shifts, coeffs
'''
def shifts_and_coeffs(params):
    """A function that defines the shift amounts and coefficients needed for
    defining a parameter-shift rule for CRX, CRY, and CRZ gates.

    Returns:
        shifts (list(float)): A list of shift amounts. Order them however you want!
        coeffs (list(float)): A list of coefficients. Order them however you want!
    """
    # Put your code here and make sure to return what is needed #
    # [1.23, 0.6, 4.56]', '[0.08144, -0.33706, -0.37944]
    
    # 0.08144 = c1*(circuit([1.23+s1, 0.6, 4.56]) - circuit([1.23-s1, 0.6, 4.56])) - c2(circuit([1.23+s2, 0.6, 4.56]) - circuit([1.23-s2, 0.6, 4.56]))
    # -0.33706 = c1*(circuit([1.23, 0.6+s1, 4.56]) - circuit([1.23, 0.6-s1, 4.56])) - c2(circuit([1.23, 0.6+s2, 4.56]) - circuit([1.23, 0.6-s2, 4.56]))
    # -0.37944 = c1*(circuit([1.23, 0.6, 4.56+s1]) - circuit([1.23, 0.6, 4.56-s1])) - c2(circuit([1.23, 0.6, 4.56+s2]) - circuit([1.23, 0.6, 4.56-s2]))
    shifts = [np.pi/2,3*np.pi/2]
    coeffs = [(np.sqrt(2)+1)/4*np.sqrt(2),(np.sqrt(2)-1)/4*np.sqrt(2)]
    
    #shifts, coeffs = train_parameters(params, shifts, coeffs)
    
    #returning [[s1,s2],[c1,c2]] ????? 
    return shifts,coeffs

def my_parameter_shift_grad(params):
    """Your homemade parameter-shift rule function!
    NOTE: you cannot use qml.grad within this function

    Args:
        params (list(float)): The parameters for gates in the circuit
    
    Returns:
        gradient (numpy.array): The gradient of the circuit with respect to the given parameters.
    """
    gradient = np.zeros_like(params)

    shifts, coeffs = shifts_and_coeffs(params)
    
    for i in range(len(params)):
        # Put your code here
        shifted = params.copy()
        shifted2 = params.copy()
        shifted3 = params.copy()
        shifted4 = params.copy()
        print(f'original param: {shifted}')
        shifted[i] = shifted[i] + shifts[0]
        shifted2[i] = shifted2[i] - shifts[0]
        
        shifted3[i] = shifted3[i] + shifts[1]
        shifted4[i] = shifted4[i] - shifts[1]
        print(f'modified param s1: {shifted}')
        print(f'modified param -s1: {shifted2}')
        print(f'modified param s2: {shifted3}')
        print(f'modified param -s2: {shifted4}')
        f1 = circuit(shifted)
        f2 = circuit(shifted2)
        forward = f1 - f2
    
        b1 = circuit(shifted3)
        b2 = circuit(shifted4)
        back = b1 - b2
    
        gradient[i] = (coeffs[0]*forward - coeffs[1]*back)/2
        print(f'gradient[{i}]: {gradient[i]}\n')
        
        
    return np.round_(gradient, decimals=5).tolist()



# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    params = json.loads(test_case_input)
    gradient = my_parameter_shift_grad(params)
    return str(gradient)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    print(f'\nsolution_output: {solution_output}')
    expected_output = json.loads(expected_output)
    print(f'expected_output: {expected_output}')
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your gradient isn't quite right!"


test_cases = [['[1.23, 0.6, 4.56]', '[0.08144, -0.33706, -0.37944]']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[1.23, 0.6, 4.56]'...
original param: [1.23, 0.6, 4.56]
modified param s1: [2.8007963267948965, 0.6, 4.56]
modified param -s1: [-0.3407963267948966, 0.6, 4.56]
modified param s2: [5.94238898038469, 0.6, 4.56]
modified param -s2: [-3.4823889803846897, 0.6, 4.56]
gradient[0]: 0.08144382176739272

original param: [1.23, 0.6, 4.56]
modified param s1: [1.23, 2.1707963267948966, 4.56]
modified param -s1: [1.23, -0.9707963267948966, 4.56]
modified param s2: [1.23, 5.312388980384689, 4.56]
modified param -s2: [1.23, -4.11238898038469, 4.56]
gradient[1]: -0.3370561075198938

original param: [1.23, 0.6, 4.56]
modified param s1: [1.23, 0.6, 6.130796326794896]
modified param -s1: [1.23, 0.6, 2.989203673205103]
modified param s2: [1.23, 0.6, 9.27238898038469]
modified param -s2: [1.23, 0.6, -0.15238898038469006]
gradient[2]: -0.3794403540904609


solution_output: [0.08144, -0.33706, -0.37944]
expected_output: [0.08144, -0.33706, -0.37944]
Correct!
